In [76]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
import math

In [77]:
def psnr(img1, img2):
    mse = np.mean( ((img1 - img2)) ** 2 )
    if mse == 0:
        return 'INF'
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


def calculatePSNR(img1):
  

    test_imgs = ["Art", "Dolls", "Reindeer"]

    gt_names = "./gt/"+test_imgs[0]+"/disp1.png";
    gt_img = np.array(Image.open(gt_names),dtype=float);

    pred_names = img1
    pred_img = np.array(Image.open(pred_names),dtype=float)

    [h,l] = gt_img.shape
    gt_img = gt_img[:, 250:l]
    pred_img = pred_img[:, 250:l]
    pred_img[gt_img==0]= 0

    peaksnr = psnr(pred_img,gt_img);
    print('The Peak-SNR value is %0.4f  ', peaksnr);

In [78]:
def get_f_r(pos, arr):
    l = r = pos
    for i in range(pos, 1, -1):
        if arr[i] > 0:
            l = i
            break
    for i in range(pos, len(arr)-1):
        if arr[i] > 0:
            r = i
            break

    return l, r


imgL = cv.imread('view1.png')
imgR = cv.imread('view5.png')

# imgL = cv.GaussianBlur(imgL, (3, 3), 0)
# imgR = cv.GaussianBlur(imgR, (3, 3), 0)

# imgL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
# imgR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)


#转换为灰度图后计算视差图

window_size = 7
min_disp = 0
num_disp = 250 - min_disp
stereo = cv.StereoSGBM_create(minDisparity = 0,
                                numDisparities = 256,
                                blockSize = window_size,
                                P1=8 * 3 * window_size ** 2,
                                P2=32 * 3 * window_size ** 2,
                                disp12MaxDiff = 1,
                                uniquenessRatio = 10,
                                speckleWindowSize = 150,
                                speckleRange = 1
                                )

disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0
new_disp = (disp - min_disp) / num_disp

count = 0
for j in range(new_disp.shape[0]):
    if new_disp[0, j] > 0:
        count = j
        break
new_disp_pp = np.zeros_like(new_disp)
for i in range(new_disp.shape[0]):
    print(i, 'running')
    for j in range(count, new_disp.shape[1] - 1):
        if new_disp[i, j] <= 0:
            left, right = get_f_r(j, new_disp[i])
            #up, down  = get_f_r(i, new_disp[:, j])
            new_disp_pp[i, j] = (new_disp[i, left - 1] + new_disp[i, right + 1]) / 2
        else:
            new_disp_pp[i, j] = new_disp[i, j]


0 running
1 running
2 running
3 running
4 running
5 running
6 running
7 running
8 running
9 running
10 running
11 running
12 running
13 running
14 running
15 running
16 running
17 running
18 running
19 running
20 running
21 running
22 running
23 running
24 running
25 running
26 running
27 running
28 running
29 running
30 running
31 running
32 running
33 running
34 running
35 running
36 running
37 running
38 running
39 running
40 running
41 running
42 running
43 running
44 running
45 running
46 running
47 running
48 running
49 running
50 running
51 running
52 running
53 running
54 running
55 running
56 running
57 running
58 running
59 running
60 running
61 running
62 running
63 running
64 running
65 running
66 running
67 running
68 running
69 running
70 running
71 running
72 running
73 running
74 running
75 running
76 running
77 running
78 running
79 running
80 running
81 running
82 running
83 running
84 running
85 running
86 running
87 running
88 running
89 running
90 running
91 runnin

In [79]:
for i in range(10):
    new_disp_pp = cv.medianBlur(new_disp_pp, 5)

In [80]:
plt.imsave('SGBM.png', new_disp_pp)
img = Image.open('SGBM.png').convert('L')
img.save('SGBM.png')
calculatePSNR("SGBM.png")

The Peak-SNR value is %0.4f   22.01550597751513


In [81]:
plt.imsave('SGBM.png', new_disp_pp)
img = cv.imread('SGBM.png')
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

for row in range(len(img)):
    for col in range(len(img[0])):
        img[row][col] += 2
cv.imwrite('SGBM.png', img)
calculatePSNR("SGBM.png")

The Peak-SNR value is %0.4f   22.033394647919955
